# Context aware pulse gate calibration with Reinforcement Learning

In this notebook, we explore the context aware optimization subroutine through a pulse level gate calibration procedure, taking an Echoed Cross Resonance (ECR) gate as a baseline gate to be optimized by the RL agent.

In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [ ]:
# Qiskit imports
from qiskit import pulse, transpile
from qiskit.circuit import (
    ParameterVector,
    QuantumCircuit,
    QuantumRegister,
    Gate,
)
from rl_qoc import QuantumEnvironment, CustomPPO, ContextAwareQuantumEnvironment
from rl_qoc.helpers import (
    get_control_channel_map,
    load_from_yaml_file,
)

from rl_qoc.config import (
    QEnvConfig,
    ActionSpace,
    BackendConfig,
    ExecutionConfig,
    GateTarget,
)
import numpy as np
from typing import Optional
import matplotlib.pyplot as plt

# Circuit macros for environment

Below we define functions defining our way of parametrizing the custom gate we intend to calibrate. In the pulse gate scenario, we define two functions for clearly separating the abstraction layers that are intertwined here.
The function ```apply_parametrized_circuit``` creates a custom parametrized ```Gate``` (which can be derived from a predefined Qiskit gate to retrieve its logical effect) and appends it to a ```QuantumCircuit``` given in input. The parametrization of the gate is provided by the ```params: ParameterVector``` argument of the function and is used to build a custom calibration of the gate. This custom calibration is defined in the second function called ```custom_schedule```, which builds a custom parametrized pulse schedule that can be attached to the gate calibration (see this tutorial for more info: https://qiskit.org/documentation/tutorials/circuits_advanced/05_pulse_gates.html).
In our approach, we design a custom schedule starting from the baseline calibration already available from IBMQ calibration results. The parameters of our custom schedule are in this framework deviations over baseline parameters of the original calibration, meant to evolve to address the particular circuit context in which the gate is going to be executed.

In [ ]:
from qiskit.providers import BackendV1, BackendV2
from qiskit_experiments.calibration_management import Calibrations
from qiskit_experiments.calibration_management import (
    FixedFrequencyTransmon,
    EchoedCrossResonance,
)
from rl_qoc.helpers import get_ecr_params


def custom_schedule(
    backend: BackendV1 | BackendV2,
    physical_qubits: list,
    params: ParameterVector,
    keep_symmetry: bool = True,
):
    """
    Define parametrization of the pulse schedule characterizing the target gate.
    This function can be customized at will, however one shall recall to make sure that number of actions match the
    number of pulse parameters used within the function (throught the params argument).
        :param backend: IBM Backend on which schedule shall be added
        :param physical_qubits: Physical qubits on which custom gate is applied on
        :param params: Parameters of the Schedule/Custom gate
        :param keep_symmetry: Choose if the two parts of the ECR tone shall be jointly parametrized or not

        :return: Parametrized Schedule
    """
    # Load here all pulse parameters names that should be tuned during model-free calibration.
    # Here we focus on real time tunable pulse parameters (amp, angle, duration)
    pulse_features = ["amp", "angle", "tgt_amp", "tgt_angle"]

    # Uncomment lines below to include pulse duration as tunable parameter
    # pulse_features.append("duration")
    duration_window = 0

    new_params, _, _, _ = get_ecr_params(backend, physical_qubits)

    qubits = tuple(physical_qubits)

    if keep_symmetry:  # Maintain symmetry between the two GaussianSquare pulses
        for sched in ["cr45p", "cr45m"]:
            for i, feature in enumerate(pulse_features):
                if feature != "duration":
                    new_params[(feature, qubits, sched)] += params[i]
                else:
                    new_params[(feature, qubits, sched)] += pulse.builder.seconds_to_samples(
                        duration_window * params[i]
                    )
    else:
        num_features = len(pulse_features)
        for i, sched in enumerate(["cr45p", "cr45m"]):
            for j, feature in enumerate(pulse_features):
                if feature != "duration":
                    new_params[(feature, qubits, sched)] += params[i * num_features + j]
                else:
                    new_params[(feature, qubits, sched)] += pulse.builder.seconds_to_samples(
                        duration_window * params[i * num_features + j]
                    )

    cals = Calibrations.from_backend(
        backend,
        [
            FixedFrequencyTransmon(["x", "sx"]),
            EchoedCrossResonance(["cr45p", "cr45m", "ecr"]),
        ],
        add_parameter_defaults=True,
    )

    # Retrieve schedule (for now, works only with ECRGate(), as no library yet available for CX)
    parametrized_schedule = cals.get_schedule("ecr", physical_qubits, assign_params=new_params)
    return parametrized_schedule

In [ ]:
# Pulse gate ansatz


def apply_parametrized_circuit(
    qc: QuantumCircuit,
    params: Optional[ParameterVector] = None,
    tgt_register: Optional[QuantumRegister] = None,
):
    """
    Define ansatz circuit to be played on Quantum Computer. Should be parametrized with Qiskit ParameterVector
    This function is used to run the QuantumCircuit instance on a Runtime backend
    :param qc: Quantum Circuit instance to add the gate on
    :param params: Parameters of the custom Gate
    :param tgt_register: Quantum Register formed of target qubits
    :return:
    """
    # qc.num_qubits
    global backend, target, action_space

    gate, physical_qubits = target.gate, target.physical_qubits
    # x_pulse = backend.defaults().instruction_schedule_map.get('x', (qubit_tgt_register,)).instructions[0][1].pulse
    if params is None:
        params = ParameterVector("theta", action_space.n_actions)
    if tgt_register is None:
        tgt_register = qc.qregs[0]

    # Choose below which target gate you'd like to calibrate
    parametrized_gate = Gate("custom_ecr", 2, params=params.params)
    # parametrized_gate = gate.copy()
    # parametrized_gate.params = params.params
    parametrized_schedule = custom_schedule(
        backend=backend, physical_qubits=physical_qubits, params=params
    )
    qc.add_calibration(parametrized_gate, physical_qubits, parametrized_schedule)
    qc.append(parametrized_gate, tgt_register)

# Definition of QuantumEnvironment

## Generic information characterizing the quantum system

The algorithm is built upon Qiskit modules. To specify how to address our quantum system of interest, we therefore adopt the IBM approach to define a quantum backend, on which qubits are defined and can be accessed via control actions and measurements.

The cell below specifies:
- ```physical_qubits```: List of qubit indices which are specifically addressed by controls , namely the ones for which we intend to calibrate a gate upon or steer them in a specific quantum state. Note that this list could include less qubits than the total number of qubits, which can be useful when one wants to take into account crosstalk effects emerging from nearest-neigbor coupling.
- ```sampling_Paulis```: number of Pauli observables  to be sampled from the system: the algorithm relies on the ability to process measurement outcomes to estimate the expectation value of different Pauli operators. The more observables we provide for sampling, the more properties we are able to deduce with accuracy about the actual state that was created when applying our custom controls. For a single qubit, the possible Pauli operators are $\sigma_0=I$, $\sigma_x=X$, $\sigma_y=Y$, $\sigma_z=Z$. For a general multiqubit system, the Pauli observables are tensor products of those single qubit Pauli operators. The algorithm will automatically estimate which observables are the most relevant to sample based on the provided target. The probability distribution from which those observables are sampled is derived from the Direct Fidelity Estimation (equation 3, https://link.aps.org/doi/10.1103/PhysRevLett.106.230501) algorithm. 
- ```N_shots```: Indicates how many measurements shall be done for each provided circuit (that is a specific combination of an action vector and a Pauli observable to be sampled)
- ```n_actions```: Indicates the number of pulse/circuit parameters that characterize our parametrized quantum circuit. For our pulse level ansatz, this number will depend on the number of parameters we are willing to tune in the original two-qubit gate calibration. 
- ```estimator_options```: Options of the Qiskit Estimator primitive. The Estimator is the Qiskit module enabling an easy computation of Pauli expectation values. One can set options to make this process more reliable (typically by doing some error mitigation techniques in post-processing). Works only with Runtime Backend at the moment
- ```abstraction_level``` chosen to encode our quantum circuit. One can choose here to stick to the usual circuit model of quantum computing, by using the ```QuantumCircuit``` objects from Qiskit and therefore set the ```abstraction_level``` to ```\"circuit\"```. However, depending on the task at hand, one can also prefer to use a pulse description of all the operations in our circuit. This is possible for both real backends and simulators by using respectively Qiskit Runtime and Qiskit Dynamics. In this case, one should define the ansatz circuit above in a pulse level fashion, and the simulation done at the Hamiltonian level, and not only via statevector calculations. In this notebook we set the ```abstraction_level``` to ```\"pulse\"```. Another notebook at the gate level is available in the repo.

In [ ]:
physical_qubits = [0, 1]
n_actions = 4
action_space = ActionSpace(n_actions, -0.5, 0.5)
sampling_Paulis = 50
N_shots = 200
abstraction_level = "pulse"
batch_size = 50

## Setting up Quantum Backend

### Simulation backend initialization: Qiskit Dynamics
If you want to run the algorithm over a simulation, one can use Qiskit Dynamics for pulse level simulation of quantum circuits. Below we set the ground for declaring a ```DynamicsBackend```.

This can be done in two ways: 

1. Declare a ```DynamicsBackend``` from a ```FakeBackend``` or ```IBMBackend``` instance and use the ```from_backend()``` method to retrieve the Hamiltonian description of such backend.
2. Alternatively, you can define your own custom Hamiltonian/Linblad that should be used to simulate the multiqubit system of interest, and feed it to a ```Solver``` instance which can be used to declare the ```DynamicsBackend```.
For more information you can check Qiskit Dynamics documentation (https://qiskit.org/documentation/dynamics/apidocs/backend.html)


In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2
from rl_qoc.helpers import get_control_channel_map
from qiskit_dynamics import DynamicsBackend, Solver
from qiskit_dynamics.array import Array
import jax

jax.config.update("jax_enable_x64", True)
# tell JAX we are using CPU
jax.config.update("jax_platform_name", "cpu")
# import Array and set default backend

Array.set_default_backend("jax")
fake_backend_v2 = FakeJakartaV2()
control_channel_map = get_control_channel_map(fake_backend_v2, physical_qubits)
dt = fake_backend_v2.target.dt
print("Coupling Map: ", list(fake_backend_v2.coupling_map.get_edges()))

In [ ]:
dynamics_options = {
    "seed_simulator": None,  # "configuration": fake_backend.configuration(),
    "control_channel_map": control_channel_map,
    "solver_options": {"method": "jax_odeint", "atol": 1e-6, "rtol": 1e-8, "hmax": dt},
    "array_library": "jax",
}
qubit_properties = fake_backend_v2.qubit_properties(physical_qubits)

# Extract channel frequencies and Solver instance from backend to provide a pulse level simulation enabling
# fidelity benchmarking
calibration_files = None

In [ ]:
# Using the from_backend method

dynamics_backend = DynamicsBackend.from_backend(
    fake_backend_v2, subsystem_list=physical_qubits, **dynamics_options
)
dynamics_backend.target.qubit_properties = qubit_properties

In [ ]:
print(dynamics_backend.coupling_map)

In [ ]:
print(qubit_properties)

## Select backend

In [ ]:
# Choose backend among the set defined above: {runtime_backend, dynamics_backend, custom_backend}
from qiskit_ibm_runtime import IBMBackend as RuntimeBackend
from rl_qoc.helpers import perform_standard_calibrations

backend = dynamics_backend
res, cals = perform_standard_calibrations(backend)
print("Selected Backend: ", backend)
if isinstance(backend, DynamicsBackend):
    print("Subsystem dims: ", backend.options.subsystem_dims)
elif isinstance(backend, RuntimeBackend):
    print("Is Backend a Simulator backend:", backend.simulator)
print("Backend options", backend.options)

In [ ]:
backend_config = BackendConfig(
    parametrized_circuit=apply_parametrized_circuit,
    backend=backend,
)
backend_config

## Define quantum target: State preparation or Gate calibration

The target of our optimal control task can be of two different types:
1.  An arbitrary quantum state to prepare with high accuracy
2. A Quantum Gate to be calibrated in a noise-robust manner

Both targets are dictionaries that are identified with a key stating their ```target_type```, which can be either ```"state"``` or ```"gate"```. The type is inferred by the target provided.

For a gate target $G$, one can add the target quantum gate with a ```"gate"``` key specifying a specific instance of a Qiskit ```Gate``` object. Here, we settle for calibrating a ```ECRGate```, which is the two-qubit basis gate of latest IBM devices.
Moreover, a gate calibration requires a set of input states $\{|s_i\rangle \}$ to be provided, such that the agent can try to set the actions such that the fidelity between the anticipated ideal target state (calculated as  $G|s_i\rangle$) and the output state are simultaneously maximized. To ensure a correlation between the average reward computed from the measurement outcomes and the average gate fidelity, the provided set of input states must be tomographically complete. Note that providing the set of ```input_states```as depicted below is optional and should be done only if you have a specific set to implement, by default it is internally set to the Pauli basis preparation stage.

For a state target, one can provide, similarly to an input state, an ideal circuit to prepare it (```"circuit": QuantumCircuit```, or a density matrix (```"dm": DensityMatrix```)..

Another important key that should figure in the dictionary is the ```"register"``` indicating the physical qubits that should be addressed by this target, i.e. upon which qubits should the target be engineered. The register can be a list of indices for qubits to be addressed in the circuit, or a ```QuantumRegister```object. If register is not provided, then by default the target register will be the list of all qubits defined up to ```Gate().num_qubits```.

In [ ]:
# Example of target gate
from qiskit.circuit.library.standard_gates import ECRGate

ECR_tgt = GateTarget(gate=ECRGate(), physical_qubits=physical_qubits)
target = ECR_tgt
print(target)

In [ ]:
from qiskit.visualization.pulse_v2 import IQXStandard

default_params, features, instructions, pulses = get_ecr_params(
    fake_backend_v2, physical_qubits=physical_qubits
)

instructions.draw(backend=fake_backend_v2, style=IQXStandard())

## Declare QuantumEnvironment object

Running the box below declares the QuantumEnvironment instance.

If selected backend is a ```DynamicsBackend```, this declaration launches a series of single qubit gate calibrations (to calibrate X and SX gate). The reason for this is that the Estimator primitive, which enables the computation of Pauli expectation values, requires calibrated single qubit gates for doing Pauli basis rotations (SX and RZ, to perform Hadamard and S gates).

In [ ]:
from rl_qoc.environment.configuration import BenchmarkConfig

# Define quantum environment
execution_config = ExecutionConfig(
    n_shots=N_shots,
    batch_size=batch_size,
    sampling_paulis=sampling_Paulis,
    c_factor=1.0,
)

benchmark_config = BenchmarkConfig(benchmark_cycle=10)
q_env_config = QEnvConfig(
    target=target,
    backend_config=backend_config,
    action_space=action_space,
    execution_config=execution_config,
    benchmark_config=benchmark_config,
)

In [ ]:
from qiskit.visualization import (
    plot_coupling_map,
    plot_circuit_layout,
    gate_map,
    plot_gate_map,
)
from qiskit.visualization.pulse_v2 import IQXDebugging

plot_gate_map(backend)

# Definition of Circuit context

Now that we have established our ```QuantumEnvironment```, we will now focus on the main research point of this paper, which is to calibrate the target gate based on its location within a specific circuit context. As we will use PyTorch to build the interface between our agent and our environment, we will wrap up our original environment within a ```TorchQuantumEnvironment``` object, which will build a suitable environment for dynamical and contextual gate calibration. But first, we define the quantum circuit in which our target operation will appear.

In [ ]:
target_circuit = QuantumCircuit(2)
target_circuit.h(0)
target_circuit.cx(0, 1)
target_circuit.x([0, 1])
target_circuit.cx(0, 1)
target_circuit.draw("mpl")

To be able to see where our ECR gate shall appear in the circuit, we have to transpile this logical circuit to the backend. To ease the visualization, we add small functions to see the circuit only on relevant physical qubits.

In [ ]:
from rl_qoc.helpers.circuit_utils import remove_unused_wires

transpiled_circ = transpile(
    target_circuit,
    backend,
    initial_layout=physical_qubits,
    basis_gates=["sx", "rz", "ecr", "x"],
    optimization_level=1,
)
remove_unused_wires(transpiled_circ).draw("mpl")

# Definition of ContextAwareQuantumEnvironment

To define the ```ContextAwareQuantumEnvironment```, we follow the Gym like definition, where we provide the structure of the observation and action spaces.

The class takes the following inputs:
- ```q_env_config: QEnvConfig```: the baseline object where the information about the backend and the target gate is
- ```circuit_context: QuantumCircuit```: The circuit in which the previously defined target operation is applied. Note that the class will automatically look for all instances of the gate within the circuit and build dedicated subcircuits (truncations) enabling the successive calibration of each gate instance. To be noted here: the gate calibration focuses only on the target qubits defined in the ```QuantumEnvironment```. In Qiskit, we typically look for the ```CircuitInstruction``` object composed of a ```Gate``` object and a set of target qubits on which the gate is applied (both defined in target).
- ```action_space```/```observation_space```: Spaces defining the range and shapes of possible actions/observations.
For now, the observation space is fixed to a set of two integers:
    - the first one indicates which random input state was selected at the beginning of the episode, so that the network have an extra information on the randomness source coming from the reward.
    - the second one indicates which instance of the gate it will calibrate. In the real-time use case, we would like the agent to generate on the fly random actions that will be applied directly for the next gate within the circuit execution. With Qiskit Runtime however, we are not able to generate those actions on the fly and will therefore load all actions associated to each instance of the gate prior to execution.


Moreover, we apply for now a sequential training loop, meaning that we will force the agent to focus on the calibration of the first gate (truncating the circuit just behind its execution) before starting to calibrate the second one (and so forth).

Since we want to run a contextual gate calibration, we need to know exactly how the circuit will be transpiled on the backend. There is therefore an internal transpilation (without any optimization) that enables the retrieval of all timings of the logical gates indicated above. We also account for the local context happening on nearest neighbor qubits on the chip.

Moreover, as we run this sequential gate calibration for each instance of the target gate within the circuit, one can check the different circuit truncations the agent will go over.

Important note: The target type of the calibration must be a quantum Gate instance (it will not work if target is a quantum state).

In [ ]:
from rl_qoc import ContextAwareQuantumEnvironment

env = ContextAwareQuantumEnvironment(
    q_env_config=q_env_config,
    circuit_context=transpiled_circ,
    intermediate_rewards=False,
)

In [ ]:
env.circuits[0].draw("mpl")

In [ ]:
env.circuits[1].draw("mpl")

In [ ]:
env.baseline_circuits[1].draw("mpl")

In [ ]:
env.estimator.options

# Definition of the Agent

In [ ]:
from pathlib import Path
import os

file_name = "agent_config.yaml"
file_location = Path(os.getcwd()) / file_name

agent_config = load_from_yaml_file(file_location)

## Hyperparameters for training

In [ ]:
from rl_qoc.agent import TotalUpdates, TrainFunctionSettings, TrainingConfig

total_updates = TotalUpdates(700)
training_config = TrainingConfig(
    training_constraint=total_updates,
    target_fidelities=[0.999, 0.9999],
    lookback_window=10,
    anneal_learning_rate=True,
    std_actions_eps=1e-3,
)

train_function_settings = TrainFunctionSettings(
    plot_real_time=True,
    print_debug=True,
    num_prints=5,
    hpo_mode=False,
    clear_history=True,
)

# Training
## Storage setup

In [ ]:
ppo_agent = CustomPPO(agent_config, env)

## Main loop

In [ ]:
ppo_agent.train(training_config=training_config, train_function_settings=train_function_settings)

In [ ]:
plt.plot(ppo_agent.training_results["circuit_fidelity"], label="Circuit Fidelity")
plt.plot(np.mean(ppo_agent.training_results["reward_history"], axis=1), label="Average Return")
plt.legend()